# Rewrite the Wigner nj symbols by means of recursions and special values

In [1]:
using JAC, SymEngine

**Note**: The Julia package `SymEngine` **is needed** in order to symbolically manipulate and simplify Racah algebra expressions in JAC **but, by default, is not automatically loaded**.


In atomic and quantum many-particle physics, (the coupling of) angular momenta and spherical tensors play a very crucial role for the evaluation and spin-angular integration of many-particle matrix elements. This coupling often leads to algebraic expressions which, more often than not, are written in terms of generalized Clebsch-Gordan coefficients and/or Wigner nj symbols as well as the Wigner rotation matrices and spherical harmonics. Although the evaluation and simplification of such expressions is, at least *in principle*, a rather straigthforward task, it may become (extremely) cumbersome if complex systems or physical scenarios are considered.

After the pioneering work by Wigner in the late 1930s, in particular Guilio Racah developed a powerful machinery in the 1940s, known as **Racah algebra (techniques)** today, in order to deal with the angular momenta of (rotationally invariant) quantum many-particle systems. Briefly speaking, this machinery includes a number of strategies to simplify (so-called) **Racah expressions**. In JAC, these expressions are defined by a summation over Wigner nj symbols of different kind(s) as well as (various integrals over) the spherical harmonics and Kronecker and triangular deltas, cf. User Guide, Section 15.1. Here, the summation may formally run over any number of angular momenta and Wigner nj symbols. Since, in the theory of angular momentum, most symbols obey a rather high symmetry, the complexity of such Racah expressions increases very rapidly as more Wigner symbols, rotation matrices *and/or* spherical harmonics appear in the (product) terms over which one needs to sum.

There are different strategies, that can be (successively) applied in order to **simplify such Racah expressions algebracially**, i.e. if the angular momenta are not specified numerically. These strategies include:

+ Use of known **special values**. This strategy replaces single Wigner nj symbols or spherical harmomic by a (much) simpler expression that, in particular, does not contain any implicit summation. In practice, each Wigner nj symbol can be analysed and perhaps replaced independently by some *special-value rule*, if available and if useful in the given context.
+ Use of **orthogonality relations** for the Wigner nj symbols or spherical harmonics.
+ Use of known algebraic **sum rules for the Wigner symbols**; in fact, the orthogonality relations can be treated as particular sum rules, and this is done also in JAC.

To make use of these strategies, we need a suitable **representation** of the various symbols from the theory of angular momenta as well as for Racah expressions as a whole. Therefore, let us first have a look for the (internal) representation of the Wigner nj symbol `W3j(ja,jb,jc,ma,mb,mc)`:

In [2]:
? W3j

search: W3j



`struct  RacahAlgebra.W3j`  ... defines a type for a Wigner 3-j symbol with symbolic arguments.

```
+ ja, jb, jc     ::Basic   ... angular momenta
+ ma, mb, mc     ::Basic   ... projections of the angular momenta above
```

---

`RacahAlgebra.W3j(ja::AngMomentum, jb::AngMomentum, jc::AngMomentum, ma::AngMomentum, mb::AngMomentum, mc::AngMomentum)`       ... constructor for defining the Wigner 3-j symbol either by Julia Symbol's or SymEngine Basic variables.


which either takes six arguments of type `Basic` (from `SymEngine`) or of the *internal* type `AngMomentum`:

In [3]:
? RacahAlgebra.AngMomentum

`struct RacahAlgebra.AngMomentum`      ... defines an (abstract) data types for symbolic angular momenta which accept the types          Basic, Symbol, Int64 and Rational{Int64} and check for being consistent with angular momenta.


Of course, very similar data types exist also for the Wigner 6j and 9j smybols:

In [4]:
? W9j

search: W9j



`struct  RacahAlgebra.W9j`  ... defines a type for a Wigner 9j symbol with symbolic arguments.

```
+ a, b, c, d, e, f, g, h, i     ::Basic   ... angular momenta
```

---

`RacahAlgebra.W9j(a::AngMomentum, b::AngMomentum, c::AngMomentum,                    d::AngMomentum, e::AngMomentum, f::AngMomentum,                   g::AngMomentum, h::AngMomentum, i::AngMomentum)`       ... constructor for defining the Wigner 9j symbol either by Julia Symbol's or SymEngine Basic variables.


However, the real **power of Racah's algebra** become apparent only, if products of these Wigner symbols appear in some summation, and if multiple *summations/integrations* can be performed algebraically. To deal with such (product) expressions or a sum of such expressions, JAC defines a (more complex struct) for a `RacahExpression` as a whole:

In [5]:
? RacahExpression

search: RacahExpression



`struct  RacahAlgebra.RacahExpression`  ... defines a type for a RacahExpression with symbolic arguments.

```
+ summations    ::Array{Basic,1}      ... Summation indices.
+ integrals     ::Array{Integral,1}   ... Integrals.
+ phase         ::Basic               ... Phase of the Racah expression.
+ weight        ::Basic               ... Weight of the Racah expression.
+ deltas        ::Array{Kronecker,1}  ... List of Kronecker deltas.
+ triangles     ::Array{Triangle,1}   ... List of Triangle deltas.
+ w3js          ::Array{W3j,1}        ... List of Wigner 3-j symbols
+ w6js          ::Array{W6j,1}        ... List of Wigner 6-j symbols
+ w9js          ::Array{W9j,1}        ... List of Wigner 9-j symbols
+ ylms          ::Array{Ylm,1}        ... List of spherical harmonics
+ djpqs         ::Array{Djpq,1}       ... List of (small) Wigner rotation matrices
```

---

`RacahAlgebra.RacahExpression()`       ... constructor for defining an empty RacahExpression.

---

`RacahAlgebra.RacahExpression(rex::RacahAlgebra.RacahExpression;`

```
    summations=..,      integrals=..,   phase=..,       weight=..,      deltas=..,     triangles=..,       
    w3js=..,            w6js=..,        w9js=..,        ylms=..,       djpqs=..) 
                
... constructor for modifying a given rex::RacahExpression by 'overwriting' the explicitly selected parts of the 
    expression.
```


As already seen from the arguments, such expressions enables one to *comprise* lists (arrays) of Wigner symbols of different types, an overall phase, weight and the formal summation over quantum numbers. Here, we shall postpone the further discussion of these `RacahExpression` but will need them to just handle the output of the recursion relations or the symmetry-representations of the Wigner symbols.



In order to make use of JAC's extension for simplifying expressions from Racah' algebra, we shall need a number of symbolic variables, whose type `Basic` is derived from `SymEngine` (although not much in-line documentation is provided by this package); in this and the two subsequent tutorials, we shall define all necessary `Basic` variables *nearby* to where we will need them:

In [6]:
? Basic

search: Basic Basics AbstractConfigurationRestriction Basis MeanFieldBasis



No documentation found.

# Summary

```
mutable struct Basic <: Number
```

# Fields

```
ptr :: Ptr{Nothing}
```

# Supertype Hierarchy

```
Basic <: Number <: Any
```


We define a number of such (Basic) variables to facilitate our later discussion:

In [7]:
a  = Basic(:a);     b  = Basic(:b);    c  = Basic(:c);   d  = Basic(:d);     ee  = Basic(:ee);    f  = Basic(:f)
X  = Basic(:X);     Y  = Basic(:Y);    Z  = Basic(:Z)  
j  = Basic(:j);     m = Basic(:m);     typeof(m)

Basic

The Wigner nj symbols are known to fullfill certain **recursion relations** which enable one to *step-up* or *step-down* in the quantum numbers. These recursion relations have been applied to make the *underlying physics* more obvious or to compute arrays of Wigner symbols more efficiently. For the Wigner 3j symbols, especially, different recursions are known and are distinguished in JAC by the (abstract) type `RacahAlgebra.AbstractRecursionW3j`:

In [8]:
? RacahAlgebra.AbstractRecursionW3j

`abstract type RacahAlgebra.AbstractRecursionW3j`      ... defines an abstract and a number of singleton types for the recursion rules for Wigner 3-j symbols.

```
+ RecursionW3jMagnetic      ... Recursion wrt. the magnetic quantum numbers.
+ RecursionW3jOneStep       ... Recursion with step-1 of the j-quantum numbers.
+ RecursionW3jHalfStep      ... Recursion with step-1/2 of the j-quantum numbers.
+ RecursionW3jLouck         ... Recursion wrt. j-quantum numbers due to Louck.
```


We can apply these recusions to any given Wigner 3j symbol:

In [9]:
ja  = Basic(:ja);    jb = Basic(:jb);    jc = Basic(:jc);    ma = Basic(:ma);    mb = Basic(:mb);    mc = Basic(:mc)
w3j = W3j(ja, jb, jc, ma, mb, mc)

W3j(ja, jb, jc; ma, mb, mc)

by just typing:

In [10]:
RacahAlgebra.recursionW3j(w3j, RacahAlgebra.RecursionW3jMagnetic())

2-element Vector{RacahExpression}:
 (-sqrt((ja - ma)*(1 + ja + ma))/sqrt((1 + jc + ma + mb)*(jc - ma - mb)))  W3j(ja, jb, jc; ma, 1 + mb, -1 + mc)  
 (-sqrt((1 + jb + mb)*(jb - mb))/sqrt((1 + jc + ma + mb)*(jc - ma - mb)))  W3j(ja, jb, jc; 1 + ma, mb, -1 + mc)  

In general, all recursion relations give rise to an array of `RacahExpression`s, though without any additional summation or other Wigner symbols; of course, we can apply any other recursion as well:

In [11]:
RacahAlgebra.recursionW3j(w3j, RacahAlgebra.RecursionW3jOneStep())

3-element Vector{RacahExpression}:
 (sqrt((jc + mc)*(-1 + jc + mc)*(1 + jb + mb)*(jb - mb))/sqrt((1 + ja + jb - jc)*(ja - jb + jc)*(-ja + jb + jc)*(1 + ja + jb + jc)))  W3j(ja, jb, -1 + jc; ma, 1 + mb, -1 + mc)  
 (2*jb*sqrt((jc + mc)*(jc - mc))/sqrt((1 + ja + jb - jc)*(ja - jb + jc)*(-ja + jb + jc)*(1 + ja + jb + jc)))  W3j(ja, jb, -1 + jc; ma, mb, mc)  
 (-sqrt((jc - mc)*(-1 + jc - mc)*(1 + jb - mb)*(jb + mb))/sqrt((1 + ja + jb - jc)*(ja - jb + jc)*(-ja + jb + jc)*(1 + ja + jb + jc)))  W3j(ja, jb, -1 + jc; ma, -1 + mb, 1 + mc)  

In [12]:
RacahAlgebra.recursionW3j(w3j, RacahAlgebra.RecursionW3jLouck())

2-element Vector{RacahExpression}:
 (-sqrt((jc + mc)*(-ja + jb + jc)*(1 + ja + jb + jc))/(sqrt(jb + mb)*(1 + 2*jc)))  W3j(ja, -1/2 + jb, -1/2 + jc; mb - mc, 1/2 - mb, -1/2 + mc)  
 (-sqrt((1 + jc - mc)*(1 + ja - jb + jc)*(ja + jb - jc)*(1 + ja + jb + jc))/(sqrt(jb + mb)*(1 + 2*jc)))  W3j(ja, -1/2 + jb, 1/2 + jc; mb - mc, 1/2 - mb, -1/2 + mc)  

If not all angular momenta (quantum numbers) of a given Wigner nj symbol are independent of each other (but defined by some *fixed* relation, for instance, $b = a + 1$), one often attempts to re-write the Wigner symbol in terms of some **special value**. In depends on the particular application of how useful such a (formal) simplification is in practice. In JAC, the *search for such special values* is facilitated by calling `RacahAlgebra.evaluate()`:

In [13]:
w3j = W3j(j+3//2, j, 3//2, m, -m-3//2, 3//2)

W3j(3/2 + j, j, 3/2; m, -3/2 - m, 3/2)

In [14]:
wa = RacahAlgebra.evaluate(w3j)

>> Special value found for  W3j(3/2 + j, j, 3/2; m, -3/2 - m, 3/2) = (-1)^(1/2 + j - m)  (sqrt((-3/2 + j - m)*(1/2 + j - m)*(-1/2 + j - m)/((1 + 2*j)*(2 + 2*j)*(3 + 2*j)*(4 + 2*j))))  .


(-1)^(1/2 + j - m)  (sqrt((-3/2 + j - m)*(1/2 + j - m)*(-1/2 + j - m)/((1 + 2*j)*(2 + 2*j)*(3 + 2*j)*(4 + 2*j))))  

The last step appears to be quite simple, as the special value of `W3j(3/2 + j, j, 3/2; m, -3/2 - m, 3/2)` can be found in many texts. Owing to the symmetry of the Wigner nj symbols, however, these *special values* are not always simple to recognize. We here note that the Wigner 3j symbols have 12 classical **equivalent forms (symmetries)**, the Wigner 6j symbols already 24, and the Wigner 9j symbols even 72 equivalent representations (symmetries), and without that their value would change if the proper phase is taken into account. These numbers of equivalent representation (symmetries) numbers do not yet account for the **Regge symmetries**, in which the quantum numbers may change by $\pm 1/2$. --- We can easily re-write all Wigner symbols by just choosing *randomly* any of the symmetric form (including the proper phase) and, then, try to simplify it again:

In [15]:
rex = RacahAlgebra.equivalentForm(w3j)
RacahAlgebra.evaluate(rex, special=true)

>> Select 10th equivalent form for W3j(3/2 + j, j, 3/2; m, -3/2 - m, 3/2)    ==>   W3j(j, 3/2 + j, 3/2; -(-3/2 - m), -m, -3/2)  
>> Special value found for  W3j(j, 3/2 + j, 3/2; -(-3/2 - m), -m, -3/2) = (-1)^(1/2 + j - m)  (sqrt((-3/2 + j - m)*(1/2 + j - m)*(-1/2 + j - m)/((1 + 2*j)*(2 + 2*j)*(3 + 2*j)*(4 + 2*j))))  .


(-1)^(1/2 + j - m)  (sqrt((-3/2 + j - m)*(1/2 + j - m)*(-1/2 + j - m)/((1 + 2*j)*(2 + 2*j)*(3 + 2*j)*(4 + 2*j))))  

In contrast to the literature, where the special values are typically shown for one standard form of the Wigner symbols (as, for instance, in the initial definition above), JAC also finds the special value for all equivalent forms as it *internally cycles through all symmetries*, and by keeping the corresponding phase and weight factors (cf. later) into account. The result of such an evaluation is always a `RacahExpression`, though it is printed in a -- more or less -- neat format.

Apart from the classical symmetries, (so-called) **Regge symmetries** are known for the Wigner 3-j and 6-j symbols and could be readily implemented into JAC, if necessary. In practice, various special values of the Wigner nj symbols are recognized also by *Mathematica* and, perhaps, by a few other computer-algebra systems. We include them here for the sake of convinience, while the **main emphasis is clearly placed upon the sum rule evaluation, a very special and convenient feature of JAC**.

A few other examples for *special values* of the Wigner 3j, 6j and 9j symbols are:

In [16]:
w3j = W3j(j, j, 0, m, -m, 0)
RacahAlgebra.evaluate(w3j)

>> Special value found for  W3j(j, j, 0; m, -m, 0) = (-1)^(j - m)  ((1 + 2*j)^(-1/2))  .


(-1)^(j - m)  ((1 + 2*j)^(-1/2))  

In [17]:
w3j = W3j(ja, jb, jc, 0, 0, 0)
RacahAlgebra.evaluate(w3j)

>> Special value found for  W3j(ja, jb, jc; 0, 0, 0) = (-1)^((1/2)*(ja + jb + jc))  (gamma(1 + (1/2)*(ja + jb + jc))*sqrt(gamma(1 + ja - jb + jc)^2*gamma(1 - ja + jb + jc)/gamma(2 + ja + jb + jc))/(gamma(1 - jc + (1/2)*(ja + jb + jc))*gamma(1 - jb + (1/2)*(ja + jb + jc))*gamma(1 - ja + (1/2)*(ja + jb + jc))))  delta(ja + jb + jc, even)  .


(-1)^((1/2)*(ja + jb + jc))  (gamma(1 + (1/2)*(ja + jb + jc))*sqrt(gamma(1 + ja - jb + jc)^2*gamma(1 - ja + jb + jc)/gamma(2 + ja + jb + jc))/(gamma(1 - jc + (1/2)*(ja + jb + jc))*gamma(1 - jb + (1/2)*(ja + jb + jc))*gamma(1 - ja + (1/2)*(ja + jb + jc))))  delta(ja + jb + jc, even)  

In [18]:
w6j = W6j( a, b, c, 3//2, c-1//2, b+1//2)
RacahAlgebra.evaluate(w6j)

>> Special value found for  W6j{a, b, c; 3/2, -1/2 + c, 1/2 + b} = (-1)^(a + b + c)  ((1/2)*((-1 + a - b + c)*(a + b - c) - 2*(2 + a + b + c)*(-a + b + c))*sqrt((a - b + c)*(1 + a + b - c)/(b*c*(1 + 2*b)*(2 + 2*b)*(3 + 2*b)*(1 + 2*c)*(2 + 2*c)*(-1 + 2*c))))  .


(-1)^(a + b + c)  ((1/2)*((-1 + a - b + c)*(a + b - c) - 2*(2 + a + b + c)*(-a + b + c))*sqrt((a - b + c)*(1 + a + b - c)/(b*c*(1 + 2*b)*(2 + 2*b)*(3 + 2*b)*(1 + 2*c)*(2 + 2*c)*(-1 + 2*c))))  

In [19]:
w6j = W6j( a, b, c, 2, c-2, b+2)
RacahAlgebra.evaluate(w6j)

>> Special value found for  W6j{a, b, c; 2, -2 + c, 2 + b} = (-1)^(a + b + c)  ((1/2)*sqrt(2)*sqrt((-3 + a - b + c)*(-2 + a - b + c)*(-1 + a - b + c)*(a - b + c)*(3 + a + b - c)*(4 + a + b - c)*(1 + a + b - c)*(2 + a + b - c)/(c*(1 + 2*b)*(2 + 2*b)*(5 + 2*b)*(3 + 2*b)*(4 + 2*b)*(1 + 2*c)*(-1 + 2*c)*(-3 + 2*c)*(-2 + 2*c))))  .


(-1)^(a + b + c)  ((1/2)*sqrt(2)*sqrt((-3 + a - b + c)*(-2 + a - b + c)*(-1 + a - b + c)*(a - b + c)*(3 + a + b - c)*(4 + a + b - c)*(1 + a + b - c)*(2 + a + b - c)/(c*(1 + 2*b)*(2 + 2*b)*(5 + 2*b)*(3 + 2*b)*(4 + 2*b)*(1 + 2*c)*(-1 + 2*c)*(-3 + 2*c)*(-2 + 2*c))))  

Alternatively, we may call this evaluation as part of a `RacahExpression` as in the following example:

In [20]:
w6j = W6j( a, b, c, 2, c-2, b-2)
rex = RacahAlgebra.equivalentForm(w6j)
RacahAlgebra.evaluate(rex, special=true)

>> Select 15th equivalent form for W6j{a, b, c; 2, -2 + c, -2 + b}    ==>   W6j{-2 + c, 2, c; b, a, -2 + b}  
>> Special value found for  W6j{-2 + c, 2, c; b, a, -2 + b} = (-1)^(a + b + c)  ((1/2)*sqrt((-1 + a + b + c)*(a + b + c)*(-2 + a + b + c)*(-3 - a + b + c)*(-2 - a + b + c)*(-1 - a + b + c)*(-a + b + c)/(b*c*(-3 + 2*b)*(-2 + 2*b)*(1 + 2*b)*(-1 + 2*b)*(1 + 2*c)*(-1 + 2*c)*(-3 + 2*c)*(-2 + 2*c))))  .


(-1)^(a + b + c)  ((1/2)*sqrt((-1 + a + b + c)*(a + b + c)*(-2 + a + b + c)*(-3 - a + b + c)*(-2 - a + b + c)*(-1 - a + b + c)*(-a + b + c)/(b*c*(-3 + 2*b)*(-2 + 2*b)*(1 + 2*b)*(-1 + 2*b)*(1 + 2*c)*(-1 + 2*c)*(-3 + 2*c)*(-2 + 2*c))))  

In [21]:
g   = Basic(:g);  h = Basic(:h)
w9j = W9j(a, b, c, d, ee, f, g, h, 0)
RacahAlgebra.evaluate(w9j)

>> Special value found for  W9j{a, b, c; d, ee, f; g, h, 0} = (-1)^(b + c + ee + f)  (((1 + 2*c)*(1 + 2*g))^(-1/2))  W6j{a, b, c; ee, d, g}  .


(-1)^(b + c + ee + f)  (((1 + 2*c)*(1 + 2*g))^(-1/2))  W6j{a, b, c; ee, d, g}  

The last example show that a Wigner 9j symbol always simplifies to a 6j symbol if one of the quantum numbers is 0. In the next tutorial we shall demonstrate how **sum rules** can be applied to more complex `RacahExpression`, the real power of *Racah's algebra*.